In [1]:
2 + 2

4

In [2]:
import os
import pyarrow as pa
import pandas as pd

In [12]:
import joblib
import xgboost as xgb

In [3]:
# Define the directory path
data_dir = os.path.join('C:\\', 'Users', 'KonuTech', 'zoomcamp-capstone-01', 'data')
data_dir

'C:\\Users\\KonuTech\\zoomcamp-capstone-01\\data'

In [4]:
# Step 4: Convert and save the combined DataFrame as a single Parquet file
combined_parquet_file = 'test_data.parquet'

In [5]:
# Step 6: EDA on the combined Parquet file
parquet_df = pd.read_parquet(os.path.join(data_dir, combined_parquet_file))  # Read the combined Parquet file

In [6]:
# Shuffle the entire DataFrame randomly
parquet_df = parquet_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [7]:
unique_clients_df = parquet_df.drop_duplicates(subset='customer_ID', keep='first')

In [8]:
unique_clients_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 924621 entries, 0 to 11363265
Data columns (total 190 columns):
 #    Column       Dtype  
---   ------       -----  
 0    customer_ID  object 
 1    S_2          object 
 2    P_2          float64
 3    D_39         float64
 4    B_1          float64
 5    B_2          float64
 6    R_1          float64
 7    S_3          float64
 8    D_41         float64
 9    B_3          float64
 10   D_42         float64
 11   D_43         float64
 12   D_44         float64
 13   B_4          float64
 14   D_45         float64
 15   B_5          float64
 16   R_2          float64
 17   D_46         float64
 18   D_47         float64
 19   D_48         float64
 20   D_49         float64
 21   B_6          float64
 22   B_7          float64
 23   B_8          float64
 24   D_50         float64
 25   D_51         float64
 26   B_9          float64
 27   R_3          float64
 28   D_52         float64
 29   P_3          float64
 30   B_10         float64


In [10]:
# unique_clients_df.to_parquet(os.path.join(data_dir, "test_data_randomly_sampled.parquet"))

In [15]:
!ls

01_prepare_train_data.ipynb
02_eda.ipynb
03_downsample_data.ipynb
04_get_champion_binary_classifier.ipynb
05_prepare_test_data.ipynb
06_score_test_data.ipynb
grid_search_results_2023-11-01_17-54-58.json
LogisticRegression_2023-11-01_17-54-58.bin
RandomForest_best_model.bin
training_log.log
XGBoost_2023-11-01_17-54-58.bin


In [35]:
model_file = "XGBoost_2023-11-01_17-54-58.bin"

In [36]:
# columns_used = ['B_16', 'B_20', 'B_3', 'B_38', 'B_7', 'B_9', 'D_44', 'D_48', 'D_55', 'D_75']
columns_used = ['B_16', 'B_19', 'B_20', 'B_23', 'B_3', 'B_38', 'B_4', 'B_7', 'B_9', 'D_44', 'D_48', 'D_55', 'D_58', 'D_74', 'D_75']
columns_used

['B_16',
 'B_19',
 'B_20',
 'B_23',
 'B_3',
 'B_38',
 'B_4',
 'B_7',
 'B_9',
 'D_44',
 'D_48',
 'D_55',
 'D_58',
 'D_74',
 'D_75']

In [37]:
def score_xgb_model(model_file, input_data, columns_used):
    # Load the XGBoost model from the specified file
    xgb_model = joblib.load(model_file)
    
    # Ensure the input DataFrame contains only the specified columns
    input_data = input_data[columns_used]
    
    # Make predictions using the loaded model
    predictions = xgb_model.predict(input_data)
    
    # Add predictions to the input data DataFrame
    input_data['prediction'] = predictions
    
    return input_data

In [38]:
df_scored = score_xgb_model(
    model_file=model_file,
    input_data=unique_clients_df,
    columns_used=columns_used
)

c:\users\konutech\zoomcamp-capstone-01\venv\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(
C:\Users\KonuTech\AppData\Local\Temp\ipykernel_24024\2377296919.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['prediction'] = predictions


In [39]:
df_scored.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 924621 entries, 0 to 11363265
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   B_16        924057 non-null  float64
 1   B_19        924057 non-null  float64
 2   B_20        924057 non-null  float64
 3   B_23        924621 non-null  float64
 4   B_3         924057 non-null  float64
 5   B_38        924057 non-null  float64
 6   B_4         924621 non-null  float64
 7   B_7         924621 non-null  float64
 8   B_9         924621 non-null  float64
 9   D_44        878448 non-null  float64
 10  D_48        804281 non-null  float64
 11  D_55        861404 non-null  float64
 12  D_58        924621 non-null  float64
 13  D_74        919718 non-null  float64
 14  D_75        924621 non-null  float64
 15  prediction  924621 non-null  int32  
dtypes: float64(15), int32(1)
memory usage: 148.6 MB


In [40]:
df_scored

,B_16,B_19,B_20,B_23,B_3,B_38,B_4,B_7,B_9,D_44,D_48,D_55,D_58,D_74,D_75,prediction
0,0.168312,0.000913,0.061406,0.020590,0.018735,2.0,0.025676,0.030808,0.021362,0.009698,0.056698,0.040742,0.003839,0.004682,0.073677,0
1,0.091521,0.005003,0.002994,0.024689,0.004492,2.0,0.048045,0.033019,0.002839,0.007198,0.249304,0.379770,0.001753,0.005654,0.006693,0
2,0.008443,0.009813,0.005902,0.002116,0.007269,2.0,0.004587,0.013267,0.002669,0.007728,0.143021,0.083083,0.004457,0.009156,0.005511,0
3,1.007958,0.912688,1.009449,0.638273,0.628909,6.0,0.364028,0.691376,0.658265,0.256353,0.971287,0.850962,0.434002,0.294366,0.338710,1
4,0.086049,0.006282,0.001018,0.065805,0.013247,2.0,0.064155,0.080140,0.162868,0.005896,0.057728,0.120027,0.003027,0.003694,0.004395,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11354522,0.006080,0.008293,0.002182,0.028284,0.009488,1.0,0.030256,0.042757,0.053617,0.004436,0.339970,0.096525,0.005186,0.002350,0.001578,0
11354756,0.009373,0.005062,0.007488,0.761858,0.003161,1.0,0.226116,0.846342,0.074930,0.006715,0.267444,0.045396,0.341347,0.576102,0.542992,1
11356280,0.006825,0.006054,0.002598,0.128480,0.002558,1.0,0.070057,0.150470,0.004941,0.004508,0.212360,0.039893,0.113015,0.078490,0.068366,0
11357428,0.253342,0.004548,0.185325,0.073859,0.309063,4.0,0.043549,0.097014,0.506914,0.009515,0.405765,0.147275,0.004282,0.008710,0.076225,1


In [41]:
# Left join df_scored to unique_clients_df on row index
merged_df = unique_clients_df["customer_ID"].to_frame().join(df_scored["prediction"].to_frame(), how='left')

In [42]:
merged_df

,customer_ID,prediction
0,f56fdbc843bc02ad4dbc20e61b493d6905126d692eec79...,0
1,cc3a057fe61c68bcc9865ca319226e2ea1a372769b3115...,0
2,376a87a3d47e34a1d40b0c026fe5a7a7c801c086b13a54...,0
3,00b513cf1a88a057c5065dc286dcb01a56a831978ea09d...,1
4,931acc1d65abc0a68d638b69ef5a3e91fda6d14bec933e...,0
...,...,...
11354522,60d63bf43d55b13117986bca0b03d6a58c182ee747af98...,0
11354756,a0aac4094d863cb7cabd0b2de32634c9fefaf814153e9e...,1
11356280,2542d893ac6a5dd0c16aa0c758e3d129f2d82b93393739...,0
11357428,4abd50915c60908fed1163b87a3ce51e5985e7bc5878e4...,1


In [45]:
merged_df.to_csv("submission_2023-11-01_17-54-58.csv", index=False)